In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

In [2]:
open_projects_df = pd.read_csv('open_projects_listing.csv')
actuals_df = pd.read_csv('actuals_data.csv')
budget_df = pd.read_csv('budget_data.csv')
planned_revenue_df = pd.read_csv('plan_revenue_data.csv')

In [3]:
# configure pandas to display floats with 2 decimal places
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
open_projects_df.columns

Index(['PROJECT_KEY', 'PROJECT_CODE', 'PROJECT_NAME', 'FUNCTIONAL_AREA',
       'PROJECT_MANAGER', 'PROJECT_CREATE_DATE', 'PROJECT_STATUS_CODES',
       'PROJECT_STATUS_LONG_NAMES'],
      dtype='object')

In [5]:
planned_revenue_df.head()  

,PROJECT_KEY,PLANNED_REVENUE
0,71906,"-21,031.20"
1,73327,"-5,375.00"
2,77798,"-8,755.00"
3,69147,"-4,011.00"
4,60395,"-286,780.64"


In [6]:
budget_df.head()

,PROJECT_KEY,PRJ_BUDGETED_REVENUE,PRJ_BUDGETED_COST
0,72068,"-4,360.00","4,000.00"
1,72076,"-138,080.00","290,068.00"
2,72310,"-31,710.00",NaN
3,72516,"-5,300.00","5,000.00"
4,72816,"-205,740.00","187,600.00"


In [7]:
actuals_df.head()

,PROJECT_KEY,END_DATE_OF_MONTH,ACCOUNT_TYPE,AMOUNT
0,78810,30/06/2025,Billed Revenue,"-9,541.00"
1,65298,30/06/2022,Billed Revenue,"-74,640.00"
2,76681,28/02/2025,Direct Costs,"10,845.58"
3,76681,31/01/2025,Direct Costs,425.39
4,75712,31/03/2025,Direct Costs,"22,138.76"


### 2. Fields to add:

1. total_proj_revenue = 
                        quoted revenue  
                        budgeted revenue (if quoted is nil)
                        billed revenue,  only if budgeted and invoiced revenue is nil (catch all)



2. LTD costs = DIRECT_COSTS_PRIOR_MTH + OVERHEADS_PRIOR_MTH + DIRECT_COSTS_CUR_MTH + OVERHEADS_CUR_MTH

3. Budgeted costs
        - from ADA
        - if blank?

4. Forecasted costs = 
        - budget + user inputted adjustment
            -> need to create a table to store user inputs to re-use in the future periods

5. Percentage_completion = LTD Costs / Forecasted Cost

6. LTD Revenue to Recognise = Percentage_completion * total_proj_revenue

7. Revenu_Recognised_LTD = Billed revenue Current Month + Billed Revenue Prior Month  + Deferred Current Mth + Deferred Prior Mth

8. MTD_Revenue_Adj = LTD Revenue to Recognise - Revenu_Recognised_LTD


### Step 1. Transform Actuals Data

In [8]:
# Create a variable that returns the last date of the month for the current month if todays date is past the 5th of the month, otherwise return the last date of the previous month
month_end = None

today = datetime.today()
if today.day > 5:
    month_end = datetime(today.year, today.month + 1, 1) - pd.Timedelta(days=1)
else:
    month_end = datetime(today.year, today.month, 1) - pd.Timedelta(days=1)

month_end = datetime(2025, 7, 31) # hardcoded for testing purposes



In [9]:
def create_projects_df(open_projects_df, actuals_df, budget_df, planned_revenue_df, month_end):
    
    open_projects_df.columns = open_projects_df.columns.str.lower()
    actuals_df.columns = actuals_df.columns.str.lower()
    budget_df.columns = budget_df.columns.str.lower()
    planned_revenue_df.columns = planned_revenue_df.columns.str.lower()

    actuals_df['end_date_of_month'] = pd.to_datetime(actuals_df['end_date_of_month'], dayfirst=True,  errors='coerce')

    prj_ltd_billed_revenue_df = (
        actuals_df[(actuals_df['account_type'] == 'Billed Revenue') & (actuals_df['end_date_of_month'] <= month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'ltd_billed_revenue'})
    )
                        
    prj_ltd_deferred_revenue_df = (
        actuals_df[(actuals_df['account_type'] == 'Deferred Revenue') & (actuals_df['end_date_of_month'] <= month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'ltd_deferred_revenue'})
    )

    prj_ltd_cost_df = (
        actuals_df[(~actuals_df['account_type'].str.contains('Revenue')) & (actuals_df['end_date_of_month'] <= month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'ltd_cost'})
    )

    prj_mtd_revenue_df = (
        actuals_df[(actuals_df['account_type'].isin(['Billed Revenue', 'Deferred Revenue'])) & (actuals_df['end_date_of_month'] == month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'mtd_revenue'})
    )

    prj_mtd_direct_cost_df = (
        actuals_df[(actuals_df['account_type'] == 'Direct Costs') & (actuals_df['end_date_of_month'] == month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'mtd_direct_cost'})
    )

    prj_mtd_overhead_cost_df = (
        actuals_df[(actuals_df['account_type'] == 'Overheads') & (actuals_df['end_date_of_month'] == month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'mtd_oh'})
    )

    prj_last_transac_mth_df = (
        actuals_df.groupby('project_key')
        .agg({'end_date_of_month': 'max'})
        .reset_index()
        .rename(columns={'end_date_of_month': 'last_transaction_month'})
    )

    projects_full_df = (
        open_projects_df[['project_key', 'project_code', 'project_name', 'project_manager', 'functional_area']]
            .merge(prj_last_transac_mth_df, on='project_key', how='left')
            .merge(prj_mtd_revenue_df, on='project_key', how='left')
            .merge(prj_mtd_direct_cost_df, on='project_key', how='left')
            .merge(prj_mtd_overhead_cost_df, on='project_key', how='left')
            .merge(prj_ltd_billed_revenue_df, on='project_key', how='left')
            .merge(prj_ltd_deferred_revenue_df, on='project_key', how='left')
            .merge(prj_ltd_cost_df, on='project_key', how='left')
            .merge(budget_df, on='project_key', how='left')
            .merge(planned_revenue_df, on='project_key', how='left')
            .fillna(0)  # Fill NaN values with 0
        )

    excluded_projects = ['NC-006914']

    projects_full_df = projects_full_df[~projects_full_df['project_code'].isin(excluded_projects)]

    return projects_full_df

def calculate_metrics(df):
    """Calculate revenue adjustments based on completion percentage and update metrics"""
    df_calc = df.copy()

    df_calc['budget_adj'] = 0
    df_calc['revenue_adj'] = 0
    # drop a specific project code from the DataFrame

    # mtd_cost = direct costs + overheads if the functional_area starts with 'L' otherwise just direct costs
    df_calc['is_capital'] = df_calc['functional_area'].str.startswith('L')
    df_calc['mtd_cost'] = df_calc['mtd_direct_cost'] + df_calc['mtd_oh'].where(df_calc['is_capital'], 0)
    df_calc['ltd_revenue'] = df_calc['ltd_billed_revenue'] + df_calc['ltd_deferred_revenue']

    # create contracted_revenue = min of ltd_billed_revenue and planned_revenue
    df_calc['contracted_revenue'] = df_calc[['ltd_billed_revenue', 'planned_revenue']].min(axis=1)

    df_calc['project_is_material'] = df_calc['contracted_revenue'] < -10_000

    df_calc['percentage_completion'] = df['ltd_cost'] / (df_calc['prj_budgeted_cost'] + df_calc['budget_adj'])

    # Replace NaN, inf, and -inf with 0, then clip between 0 and 1
    df_calc['percentage_completion'] = (df_calc['percentage_completion']
                                        .replace([np.inf, -np.inf], 0)
                                        .fillna(0)
                                        .clip(0, 1)
                                        )

    df_calc['percentage_completion'] = df_calc['percentage_completion'].where(df_calc['project_is_material'], 1)

    df_calc['revenue_adj'] = (
        df_calc['contracted_revenue'] * df_calc['percentage_completion'] - (df_calc['ltd_billed_revenue'] + df_calc['ltd_deferred_revenue'])
    )

    return df_calc

In [10]:
projects_full_df = create_projects_df(open_projects_df, actuals_df, budget_df, planned_revenue_df, month_end)

In [11]:
projects_full_df.columns

Index(['project_key', 'project_code', 'project_name', 'project_manager',
       'functional_area', 'last_transaction_month', 'mtd_revenue',
       'mtd_direct_cost', 'mtd_oh', 'ltd_billed_revenue',
       'ltd_deferred_revenue', 'ltd_cost', 'prj_budgeted_revenue',
       'prj_budgeted_cost', 'planned_revenue'],
      dtype='object')

In [12]:
modified_prj_df = calculate_metrics(projects_full_df)

In [13]:
modified_prj_df

,project_key,project_code,project_name,project_manager,functional_area,last_transaction_month,mtd_revenue,mtd_direct_cost,mtd_oh,ltd_billed_revenue,...,prj_budgeted_cost,planned_revenue,budget_adj,revenue_adj,is_capital,mtd_cost,ltd_revenue,contracted_revenue,project_is_material,percentage_completion
0,75010,NC-024454,ISA NWP Oakden Hills PS 7.04MW,ROSITANO Rocco,L03,2025-06-30 00:00:00,0.00,0.00,0.00,"-140,610.00",...,0.00,"-140,610.00",0,"53,143.99",True,0.00,"-53,143.99","-140,610.00",True,0.00
1,73231,NC-023400,AR-13-15-Colley Tce-Glenelg,AYYAPPAN Sivaranjani,S42,2025-06-30 00:00:00,0.00,0.00,0.00,"-87,000.00",...,"86,900.00","-87,000.00",0,0.00,False,0.00,"-87,000.00","-87,000.00",True,1.00
2,70857,NC-021987,"RC - Nuriootpa Traders, Nuriootpa",GUSLING Daniel,L03,2025-07-31 00:00:00,0.00,-529.38,-133.40,"-464,410.00",...,"293,206.00","-464,410.00",0,-246.72,True,-662.78,"-464,163.28","-464,410.00",True,1.00
3,76416,NC-025328,EG1342024_CONEYBEER STREET_BERRI,BERNSEE Carlos,S17,2025-07-31 00:00:00,0.00,"1,627.39",255.50,"-4,580.00",...,"4,580.00","-4,580.00",0,0.00,False,"1,627.39","-4,580.00","-4,580.00",False,1.00
4,65298,NC-018977,"RC - OTR, Virginia",CADDY Tim,L03,2023-09-30 00:00:00,0.00,0.00,0.00,"-93,300.00",...,"266,300.00","-93,300.00",0,"2,187.35",True,0.00,"-93,300.00","-93,300.00",True,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,77337,NC-025838,"AR-ScottSalisbury-22A BeyerSt,Norwood",SAION Norlina,S42,2025-07-31 00:00:00,0.00,502.26,78.85,"-31,590.00",...,"30,303.00","-31,590.00",0,"11,011.81",False,502.26,"-31,590.00","-31,590.00",True,0.65
2256,70829,NC-021977,"RD - 2 Mount Terrace, Gawler",ALI Uzma,L43,2025-01-31 00:00:00,0.00,0.00,0.00,"-94,590.00",...,"79,603.00","-94,590.00",0,0.00,True,0.00,"-93,335.59","-94,590.00",True,0.99
2257,69986,NC-021566,"RC-STEBONHEATH ROAD, EDINBURGH NORTH",GUSLING Daniel,L03,2024-07-31 00:00:00,0.00,0.00,0.00,"-240,110.00",...,"233,200.00","-240,110.00",0,0.00,True,0.00,"-240,110.00","-240,110.00",True,1.00
2258,77265,NC-025813,"EG1353009 - 100kVA - 29 Young Street, Ad",DEBRA Kanon,S17,2025-04-30 00:00:00,0.00,0.00,0.00,"-5,520.00",...,0.00,"-5,520.00",0,0.00,False,0.00,"-5,520.00","-5,520.00",False,1.00


In [ ]:
def calculate_margin_exceptions(df, margin_threshold = 0.3):
    # Identify projects with margin below the threshold
    display_df = df.copy()
    display_df['contracted_revenue'] = display_df[['ltd_billed_revenue', 'planned_revenue']].min(axis=1)
    display_df['contracted_revenue'] = -display_df['contracted_revenue']
    display_df['prj_budgeted_revenue'] = -display_df['prj_budgeted_revenue']
    display_df['planned_margin'] = (display_df['contracted_revenue'] - display_df['prj_budgeted_cost']) / display_df['prj_budgeted_cost']
    display_df['planned_margin'] = (display_df['planned_margin'].fillna(0)
                                                                .replace([np.inf, -np.inf], 0) 
                                    )
    display_df['project_is_material'] = display_df['contracted_revenue'] < -10_000

    display_df = display_df[(display_df['planned_margin'].abs() > margin_threshold) & (display_df['project_is_material'])]
    
    display_df = display_df[['project_code', 'project_name', 'functional_area', 'contracted_revenue', 'prj_budgeted_revenue','prj_budgeted_cost', 'planned_margin']]

    return display_df

In [ ]:
calculate_margin_exceptions(projects_full_df)

,project_code,project_name,functional_area,contracted_revenue,prj_budgeted_revenue,prj_budgeted_cost,planned_margin
2,NC-021987,"RC - Nuriootpa Traders, Nuriootpa",L03,"464,410.00","177,610.00","293,206.00",0.58
4,NC-018977,"RC - OTR, Virginia",L03,"93,300.00","93,300.00","266,300.00",-0.65
6,NC-022252,"RD-Pederick Road, Lewiston",L04,"25,324.00","82,360.00","90,890.00",-0.72
15,NC-023602,RD RIVERLEA STAGE 39A,L04,"233,375.00","-70,840.00","67,067.00",2.48
16,CP-013986,PLEC Chief St Brompton,L33,"1,046,871.00","1,046,871.00","3,150,613.00",-0.67
...,...,...,...,...,...,...,...
2229,NC-020969,EG BE 4.95MVA PVBESS Moyhall,L03,"360,941.00","345,470.00","138,800.00",1.60
2233,NC-019749,RD - Kapunda Kidman Stage Rise 1D,L04,"27,723.00","5,200.00","13,733.00",1.02
2247,NC-024200,Mount Barker Hospital,L03,"86,514.00","409,370.00","705,346.00",-0.88
2253,NC-023401,RC-TELSTRA-OAKDEN HILLS,L03,"22,530.00","22,530.00","104,122.00",-0.78


In [49]:
ranked_projects = modified_prj_df.copy()
ranked_projects['display_revenue'] = -(ranked_projects['mtd_revenue'] + ranked_projects['revenue_adj'])
ranked_projects['max_mtd_value'] = ranked_projects[['display_revenue', 'mtd_cost']].max(axis=1)
ranked_projects = ranked_projects.nlargest(10, 'max_mtd_value')

In [50]:
ranked_projects

,project_key,project_code,project_name,project_manager,functional_area,last_transaction_month,mtd_revenue,mtd_direct_cost,mtd_oh,ltd_billed_revenue,...,budget_adj,revenue_adj,is_capital,mtd_cost,ltd_revenue,contracted_revenue,project_is_material,percentage_completion,display_revenue,max_mtd_value
1480,62070,NC-016820,NC: FLINDERS PORT UPGRADED 28MVA SUPPLY,CRICHTON Ann,L03,2025-07-31 00:00:00,0.00,"93,762.75","23,342.84","-7,563,360.91",...,0,"-6,834,698.18",True,"117,105.59","-728,662.73","-7,563,360.91",True,1.00,"6,834,698.18","6,834,698.18"
861,67594,NC-020280,SA Water Happy Valley Sub Upgr,ROSITANO Rocco,L03,2025-07-31 00:00:00,0.00,295.74,74.53,"-2,611,350.00",...,0,"-2,145,339.00",True,370.27,"-466,011.00","-2,611,350.00",True,1.00,"2,145,339.00","2,145,339.00"
1316,76705,NC-025499,ANI 11kV Link Rd Relocations,BARONE Adrian,S42,2025-07-31 00:00:00,0.00,"330,976.15","83,291.70","-1,803,700.00",...,0,"-1,507,852.73",False,"330,976.15","-295,847.27","-1,803,700.00",True,1.00,"1,507,852.73","1,507,852.73"
1442,73984,NC-023835,"DIT - Raglan Ave, Edwardstown",CULIBRK Sinisa,S42,2025-07-31 00:00:00,0.00,"5,313.13","1,279.10","-3,162,110.00",...,0,"-1,218,559.34",False,"5,313.13","-94,972.67","-3,162,110.00",True,0.42,"1,218,559.34","1,218,559.34"
1816,50880,NC-009977,Submarine Osborne 2x20MVA,CULIBRK Sinisa,L03,2025-06-30 00:00:00,0.00,0.00,0.00,"-6,377,720.00",...,0,"-1,208,644.30",True,0.00,"-168,411.00","-6,377,720.00",True,0.22,"1,208,644.30","1,208,644.30"
827,73974,NC-023832,SA Water EP Desal Plant,ROSITANO Rocco,L03,2025-07-31 00:00:00,0.00,"655,866.79","166,966.86","-10,478,463.00",...,0,"-627,982.75",True,"822,833.65","-2,702,860.27","-10,612,163.00",True,0.31,"627,982.75","822,833.65"
1485,72352,NC-022881,RC Baiada Poultry Tarlee,MENADUE Mark,L03,2025-07-31 00:00:00,0.00,"641,941.92","161,758.09","-806,730.02",...,0,"-774,997.21",True,"803,700.01","-31,732.81","-806,730.02",True,1.00,"774,997.21","803,700.01"
920,71003,NC-022082,DIT MTX UG 66kV OH Line,ASAD Muhammad,S42,2025-07-31 00:00:00,0.00,"377,525.20","94,999.58","-5,718,040.00",...,0,"-722,245.27",False,"377,525.20","-3,676,201.45","-5,718,040.00",True,0.77,"722,245.27","722,245.27"
1202,66750,NC-019764,RAAF & DST Supply Upgrade 66MW,ROSITANO Rocco,L03,2025-07-31 00:00:00,0.00,"388,276.06","158,318.70","-12,673,340.00",...,0,"-657,908.04",True,"546,594.76","-2,015,821.32","-12,673,340.00",True,0.21,"657,908.04","657,908.04"
1839,75164,NC-024542,Cultana East ENET Station Sup,ROSITANO Rocco,L03,2025-07-31 00:00:00,0.00,"408,275.93","103,322.97","-1,153,680.00",...,0,"-597,077.65",True,"511,598.90","-301,969.43","-1,153,680.00",True,0.78,"597,077.65","597,077.65"


In [51]:
def create_projects_df(open_projects_df, actuals_df, budget_df, month_end):

    open_projects_df.columns = open_projects_df.columns.str.lower()
    actuals_df.columns = actuals_df.columns.str.lower()
    budget_df.columns = budget_df.columns.str.lower()

    actuals_df['end_date_of_month'] = pd.to_datetime(actuals_df['end_date_of_month'], dayfirst=True,  errors='coerce')

    prj_ltd_cost_df = (
        actuals_df[(~actuals_df['account_type'].str.contains('Revenue')) & (actuals_df['end_date_of_month'] <= month_end)]
        .groupby('project_key')
        .agg({'amount': 'sum'})
        .reset_index()
        .rename(columns={'amount': 'ltd_cost'})
    )

    prj_last_transac_mth_df = (
        actuals_df.groupby('project_key')
        .agg({'end_date_of_month': 'max'})
        .reset_index()
        .rename(columns={'end_date_of_month': 'last_transaction_month'})
    )

    projects_full_df = (
        open_projects_df[['project_key', 'project_code', 'project_name', 'project_manager', 'functional_area']]
            .merge(prj_last_transac_mth_df, on='project_key', how='left')
            .merge(prj_ltd_cost_df, on='project_key', how='left')
            .merge(budget_df, on='project_key', how='left')
            .fillna(0)  # Fill NaN values with 0
        )

    excluded_projects = ['NC-006914']

    projects_full_df = projects_full_df[~projects_full_df['project_code'].isin(excluded_projects)]

    return projects_full_df

def calculate_exceptions(df, cost_threshold = 100_000, budget_threshold = 0.3):
    # Identify projects with revenue below the threshold
    df['budget_error'] = ((df['ltd_cost'] > cost_threshold) & (df['prj_budgeted_cost'] / df['ltd_cost'] < budget_threshold))
    return df[df['budget_error']]

In [52]:
df = create_projects_df(open_projects_df, actuals_df, budget_df, month_end)

In [54]:
df = calculate_exceptions(df)
df

,project_key,project_code,project_name,project_manager,functional_area,last_transaction_month,ltd_cost,prj_budgeted_revenue,prj_budgeted_cost,budget_error
187,73206,NC-023388,CMAR - Central Markets Stage 2,ASAD Muhammad,L03,2025-07-31 00:00:00,"1,635,999.25",0.00,0.00,True
421,70918,NC-022024,DIT MAC P4 11kV Supplies,CRICHTON Ann,L03,2025-07-31 00:00:00,"543,314.03","-773,400.00",0.00,True
1099,61802,NC-016655,AR - N-S PACKAGE 5- 394 SOUTH RD,CULIBRK Sinisa,S42,2025-07-31 00:00:00,"116,054.83",0.00,0.00,True
1110,59916,NC-015496,RC - Central Markets Redevelopment,ASAD Muhammad,L03,2025-07-31 00:00:00,"133,178.49",0.00,0.00,True
1139,74875,NC-024376,DIT - MTX 66KV TRAM UNDERBORE,ASAD Muhammad,S42,2025-07-31 00:00:00,"684,873.04",0.00,"5,000.00",True
1316,76705,NC-025499,ANI 11kV Link Rd Relocations,BARONE Adrian,S42,2025-07-31 00:00:00,"710,340.06","-24,400.00","19,419.00",True
1578,66363,NC-019538,T2D - Northern Supply Point,CULIBRK Sinisa,L03,2025-07-31 00:00:00,"1,950,770.90",0.00,0.00,True
1696,75078,NC-024489,DIT-Waterloo Cnr Precast Yard,BARONE Adrian,L03,2025-07-31 00:00:00,"607,667.86",0.00,0.00,True
1732,72051,NC-022673,Wave Generation Pt Willunga,BARONE Adrian,L03,2025-06-30 00:00:00,"134,031.72","-25,240.00","24,845.00",True
1761,72056,NC-022676,RAAF & DST Supply Upgrade 34MV,ROSITANO Rocco,L03,2025-07-31 00:00:00,"182,387.32",0.00,0.00,True
